<a href="https://colab.research.google.com/github/eiki-code/Text_cls/blob/main/BERT_matome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.26.0
!pip install datasets==2.10.1 fugashi==1.2.1 ipadic==1.0.0

In [ ]:
from google.colab import drive

drive.mount("/content/drive/")

base_path = "/content/drive/My Drive/bert_book/"

Mounted at /content/drive/


In [ ]:
import glob  # ファイルの取得に使用
import os

text_path = base_path + "text/"  # フォルダの場所を指定

dir_files = os.listdir(path=text_path)  # ファイルとディレクトリ一覧
dirs = [f for f in dir_files if os.path.isdir(os.path.join(text_path, f))]  # ディレクトリ一覧

text_label_data = []  # 文章とラベルのセット
dir_count = 0  # ディレクトリ数のカウント
file_count= 0  # ファイル数のカウント

for i in range(len(dirs)):
    dir = dirs[i]
    files = glob.glob(text_path + dir + "/*.txt")  # ファイルの一覧
    dir_count += 1

    for file in files:
        if os.path.basename(file) == "LICENSE.txt":
            continue

        with open(file, "r") as f:
            text = f.readlines()[3:]  # 先頭の3行を除去
            text = "".join(text)  # リストを文字列に変換
            text = text.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""}))  # 特殊文字を除去
            text_label_data.append([text, i])

        file_count += 1

print("\rfiles: " + str(file_count) + "dirs: " + str(dir_count), end="")

files: 7697dirs: 9

In [ ]:
import csv
from sklearn.model_selection import train_test_split

train_data, test_data =  train_test_split(text_label_data, shuffle=True)  # 訓練用とテスト用に分割
csv_path = base_path + "csv/"

if not os.path.exists(csv_path):  # ディレクトリが存在しなければ
    os.makedirs(csv_path)  # ディレクトリを作成

with open(csv_path+"train_data.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(train_data)

with open(csv_path+"test_data.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(test_data)

In [ ]:
from transformers import BertForSequenceClassification, BertJapaneseTokenizer

sc_model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese", num_labels=9)
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
model_path = base_path + "model/"

if not os.path.exists(model_path):  # ディレクトリが存在しなければ
    os.makedirs(model_path)  # ディレクトリを作成

sc_model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/My Drive/bert_book/model/tokenizer_config.json',
 '/content/drive/My Drive/bert_book/model/special_tokens_map.json',
 '/content/drive/My Drive/bert_book/model/vocab.txt',
 '/content/drive/My Drive/bert_book/model/added_tokens.json')

In [ ]:
from datasets import load_dataset

train_data = load_dataset("csv", data_files=csv_path+"train_data.csv", column_names=["text", "label"], split="train")
train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "label"])

eval_data = load_dataset("csv", data_files=csv_path+"test_data.csv", column_names=["text", "label"], split="train")
eval_data = eval_data.map(tokenize, batched=True, batch_size=len(eval_data))
eval_data.set_format("torch", columns=["input_ids", "label"])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0ce6f7b080c4ca91/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


Map:   0%|          | 0/5772 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-45d95983f0b89ad1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


Map:   0%|          | 0/1925 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "./results",  # 結果を格納するディレクトリ
    logging_dir = "./logs",  # 途中経過のログを格納するディレクトリ
    num_train_epochs = 2,  # エポック数
    per_device_train_batch_size = 8,  # 訓練時のバッチサイズ
    per_device_eval_batch_size = 32,  # 評価時のバッチサイズ
    warmup_steps=500,  # 学習系数がこのステップ数で徐々に増加
    weight_decay=0.01,  # 重みの減衰率
    evaluation_strategy = "steps"  # 訓練中、一定のステップごとに評価
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model = sc_model,  # 使用するモデルを指定
    args = training_args,  # TrainingArgumentsの設定
    compute_metrics = compute_metrics,  # 評価用の関数
    train_dataset = train_data,  # 訓練用のデータ
    eval_dataset = eval_data  # 評価用のデータ
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1925
  Batch size = 32


{'eval_loss': 0.20916110277175903,
 'eval_accuracy': 0.9496103896103896,
 'eval_runtime': 63.4041,
 'eval_samples_per_second': 30.361,
 'eval_steps_per_second': 0.962,
 'epoch': 2.0}

In [ ]:
model_path = base_path + "model/"

if not os.path.exists(model_path):  # ディレクトリが存在しなければ
    os.makedirs(model_path)  # ディレクトリを作成

sc_model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
loaded_model = BertForSequenceClassification.from_pretrained(model_path)
loaded_tokenizer = BertJapaneseTokenizer.from_pretrained(model_path)

In [ ]:
import glob  # ファイルの取得に使用
import os
import torch

category = "smax"
files = glob.glob(text_path + category + "/*.txt")  # ファイルの一覧
file = files[12]  # 適当なニュース

dir_files = os.listdir(path=text_path)
dirs = [f for f in dir_files if os.path.isdir(os.path.join(text_path, f))]  # ディレクトリ一覧

with open(file, "r") as f:
    sample_text = f.readlines()[3:]  # 先頭の3行を除去
    sample_text = "".join(sample_text)  # リストを文字列に変換
    sample_text = sample_text.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""}))  # 特殊文字を除去

print(sample_text)

max_length = 512
words = loaded_tokenizer.tokenize(sample_text)
word_ids = loaded_tokenizer.convert_tokens_to_ids(words)  # 単語をIDに変換
x = torch.tensor([word_ids[:max_length]])  # テンソルに変換

y = loaded_model(x)  # 予測
pred = y[0].argmax(-1)  # 最大値のインデックス
print("result:", dirs[pred])

L-06D JOJOの価格はいくらなのか！？ 昨日18日（土）からドコモショップなどで事前予約が開始された今夏のNTTドコモ「2012年夏モデル」のうちで人気マンガ「ジョジョの奇妙な冒険」の連載25周年を記念した限定モデルで通称“ジョジョスマホ”とも言われる「JOJO L-06D」（LGエレクトロニクス製）。予約が開始される18日朝を前にして、前日の夕方頃から店頭前に並びはじめるという徹夜組も出たほどの過熱ぶりで、コアなファンの熱い思いが垣間見られ、公式Twitterアカウント「@jojo_sma」による予約争奪戦に参加した人々のツイートの模様は見ているほうもドキドキッしてしまいました。そんな限定数1万5千台が販売されるL-06D JOJOですが、今回は、発売予定日の8月30日（木）における購入に向けて気になる価格について紹介しておこうと思います。ドコモショップで案内された価格は以下の通り。ただし、店舗によって特典などがあり、違いなどがあるかと思いますので、実際に予約したり、購入したりする店舗で聞いてみてください。■新規本体価格：84,000円月々サポート：2,520円×24回（総額60,480円）実質負担額：23,520円■機種変更本体価格：84,000円月々サポート：2,520円×24回（総額60,480円）実質負担額：23,520円■MNP（新規）本体価格：84,000円月々サポート：3,990円×24回（総額95,760円）実質負担額：-11,760円まず、本体価格は限定モデルだけあってハイエンドモデルと比べても高く、ベースモデルのOptimus Vu L-06Dと比べても1,500円ほど高くなっています。1,500円ならいいかというか、是非、ジョジョスマホのほうが欲しいと思ってしまいますね。また、月々サポートを適用させた実質負担額になっても、Optimus Vu L-06Dより2,500円ほど高くなっています。ただし、MNPによる実質負担価格については同額で、他のハイエンドモデルと比べても同価格帯となっているため、月々サポートが多く設定されており、手を出しやすくなっている印象です。なお、各店舗施策としてキャンペーンやキャッシュバックなどがある場合もありますので、実質価格はさらに下がることもあるのかなと思われます。記事執筆：memn0ck■関連リンク・エスマ

In [ ]:
print(pred)
print(dirs)

tensor([1])
['movie-enter', 'smax', 'sports-watch', 'dokujo-tsushin', 'kaden-channel', 'livedoor-homme', 'it-life-hack', 'peachy', 'topic-news']


うまくいった
